# Etude du problème d'optimisation


Question 1:
Sur l'intervalle $$\begin{align} [t_i,t_{+1}]\end{align}$$ on cherche à maximiser l'énergie autoconsommée. 
Or la consommation d'énergie autoconsommée peut être séparée en deux cas: 
1er cas: On consomme toute l'énergie des panneaux solaires soit $E_iPV$

2eme cas: On consomme moins que l'énergie produite par les panneaux, soit toute l'énergie nécessaire à la maison est fournie par les panneaux soit $w_i+ P_idt$
Ainsi, l'énergie autoconsommée est le min de ces deux valeurs
On retrouve bien la formule

On peut considérer le problème autrement en considérant que les consommations non pilotables sont nécessairement fournies par les panneaux (on cionsidere l'énergie auto consommée en négatif si il les panneaux ne suffisent pas à répondre ). On cherche alors a minimiser la fonction donnéee par l'énoncé.

Ce cout est difficile à optimiser car la fonction min n'est pas continue, ainsi la fonction que l'on cherche à optimiser n'est pas continue ni dérivable sur son intervalle de définition. Ainsi, les algorithmes d'optimisation ne sont alors pas applicables.

# Etude et résolution du problème de softmin.

On peut choisir d'utiliser les algorithmes qui s'appliquent à des fonctions continues et différentiables sous contraintes tels que Casadi, Arrow-Hurwitz ou optimise.minimize de Scipy.


On utilise Casadi

In [25]:
from casadi import *
import numpy as np
opti = casadi.Opti()
t_0=6
t_f=19
dt =  0.25
n = int((t_f -t_0)/dt)
alpha = 100
k= 0.2
T_sat= 70
T_f=70
T_in=50
C = 100
P_M= 3000
P = opti.variable((2*n))
f = 0
Q=np.zeros(n)
Q[11]=3

def h(x,y):
    return (x*exp(-alpha*x)+ y*exp(-alpha*y))/(exp(-alpha*x)+exp(-alpha*y))

for i in range(n):
    t = t_0 + i*dt
    E = 2*exp(-(t-13)**2/9)
    f+= -h(E,P[i]*dt)


opti.minimize(f)
for i in range(n):
    opti.subject_to(P[i]-P_M<=0)
    if i!=n-1:
        opti.subject_to((P[n+i+1]- exp(-k*dt)*P[i]-C/k*(1-exp(-k*dt))*(-Q[i]+P[i]))==0)
    opti.subject_to(P[n+i]<=T_sat)
opti.subject_to(P[n-1]==0)
opti.subject_to(P[2*n-1]==T_f)
x0 = np.zeros((2*n))
opti.set_initial(P,x0)
opti.solver('ipopt')
sol = opti.solve()
print(opti.debug.value(P)[0:n])

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      104
Number of nonzeros in inequality constraint Jacobian.:      104
Number of nonzeros in Lagrangian Hessian.............:       52

Total number of variables............................:      104
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       53
Total number of inequality constraints...............:      104
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      104

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

opti15_x_1


# Etude et résolution numérique du problème avec variable slack

Les variables slack permettent de transformer des contraintes inégalités en contraintes égalités. Ainsi, le problème se réécrit alors: min
x
f(x)
tel que ceq(x) = 0,
cin(x) ≤ 0

avec x=[P0,   PN, T0    TN,s0, sN]
On alors pour les contraintes égalités:
Pour les contraintes inégalités:

In [ ]:
Le problème semble désormais infaisable, surement un problème de code

In [24]:
opti = casadi.Opti()
t_0=6
t_f=19
dt =  0.25
n = int((t_f -t_0)/dt)
alpha = 100
k= 0.2
T_sat= 70
T_f=70
T_in=50
C = 100
P_M= 3
P = opti.variable(3*n+1)
f = 0
Q=np.zeros(n)
Q[-1]=3

def h(x,y):
    return (x*exp(-alpha*x)+ y*exp(-alpha*y))/(exp(-alpha*x)+exp(-alpha*y))

for i in range(n):
    t = t_0 + i*dt
    E = 2*exp(-(t-13)**2/9)
    f+= -h(E,P[i]*dt)


opti.minimize(f)
for i in range(n):
    opti.subject_to(P[2*n+i]-P_M==0)
    opti.subject_to(P[i]>=P[2*n+i])
    t = t_0 + i*dt
    E = 2*exp(-(t-13)**2/9)
    opti.subject_to(E>=P[2*n+i])
    if i<n-1:
        opti.subject_to((P[n+i+1]- exp(-k*dt)*P[i]-C/k*(1-exp(-k*dt))*(-Q[i]+P[i]))==0)
    opti.subject_to(P[n+i]<=T_sat)
opti.subject_to(P[n-1]==0)
opti.subject_to(P[2*n-1]==T_f)

x0 = np.zeros(3*n+1)
opti.set_initial(P,x0)
opti.solver('ipopt')
sol = opti.solve()
print(sol.value(P))

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      156
Number of nonzeros in inequality constraint Jacobian.:      208
Number of nonzeros in Lagrangian Hessian.............:       52

Total number of variables............................:      157
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      105
Total number of inequality constraints...............:      156
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      156

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/optistack_internal.cpp:999: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Infeasible_Problem_Detected'